
![alt text](https://www.business.unsw.edu.au/style%20library/asb/assets/images/logo-unsw.png)
__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
#from google.colab import files
import statsmodels.formula.api as smf 
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()
import datetime
%matplotlib inline 

In [2]:
from p_values import Variance_Inflation_Factor
def variance_inflation_factor(x):
    columns = x.columns
    VIF = Variance_Inflation_Factor(x)
    df_vif = pd.DataFrame({"vif": VIF})
    df_vif = df_vif.set_index(columns)
    return df_vif

In [3]:
import calendar
import re
import pandas as pd, numpy as np
from datetime import date

In [4]:
#for plotting
Data6 = pd.read_excel("20201123-tv_smvariables.xlsx").set_index("short_code")

In [5]:
numword = [x for x in Data6.columns if re.search("num",x)]

In [69]:

y = Data6['log_likes']
model_31 = ["faces","Physical_Hor_Picture", "Diagonal_dominance_Picture", "Warm Hue_Picture"]
model_32 = ["dummy_motivation_event","ikes-1day","faces","Physical_Hor_Picture", "Diagonal_dominance_Picture", "Warm Hue_Picture"]
robust_1 = ["dummy_motivation_event","ikes-1day","faces","Physical_Hor_Picture","len_text_total",
            "Diagonal_dominance_Picture", "Warm Hue_Picture","Brightness_Picture","Colorfulness_Picture","Contrast_Picture_z",
           ] + numword
robust_2 = ["dummy_motivation_event","ikes-1day","faces","Physical_Hor_Picture",
            "Diagonal_dominance_Picture", "Warm Hue_Picture","Brightness_Picture","Colorfulness_Picture","Contrast_Picture_z",
           ] 
x = Data6.drop(columns=['log_likes',"happiness",
                        ]+model_32)
x = sm.add_constant(x)
#covtype = 'HC0' for sandwich regression
model= sm.OLS(y, x).fit(cov_type='HC0')

# sandwich regression log(y) = b0 + log(x)

In [70]:
x_log = x.copy()
for col in ["product_only","product_model","model_only"]:
    x_log[col] = x_log[col].apply(lambda x:np.log1p(x))
model= sm.OLS(y, x_log).fit(cov_type='HC0')

In [56]:
#export for sales regression:
#for_sales = Data6.drop(["redudancy","temp","numFocalPd","product_type","Contrast_Picture",'comment','log_comment','post_age',"smile"]+log_IV, axis=1).astype(float)
#for_sales.to_excel("20200925_ivariables.xlsx")

In [57]:
#for_sales.info(max_cols=2000)

# execution 

In [71]:
vif = variance_inflation_factor(x)

df_coeffs = pd.DataFrame({"Coefficients": model.params,"Std_error":model.bse, "p": model.pvalues, "vif": vif["vif"]})
df_coeffs_sorted = df_coeffs.reindex(df_coeffs["Coefficients"].sort_index(ascending=False).index)
dummy_col = [x for x in df_coeffs.T.columns if re.search("dummy",x) and not(re.search("motivation",x))]
regression_vif = df_coeffs.T.drop(columns = dummy_col).T.sort_values(["vif"], axis=0, ascending=False).style.bar(subset = ["Coefficients", "p", "vif"], align='mid', color=['#d65f5f', '#5fba7d'])

In [72]:
tidy_index = list(pd.read_excel("index_smanalysis.xlsx").dropna()["var"])
final_idx = tidy_index.copy()

In [73]:
counter = 0
for i,idx in enumerate(tidy_index):
    try:
        #print(i)
        df_coeffs.T.drop(columns = dummy_col).T.loc[idx]
    except:
        print(idx)
        final_idx.pop(i-counter)
        counter = counter+1
        
#len(final_idx)==len(tidy_index)
if (len(list(df_coeffs.T.drop(columns = dummy_col).T.index)) == len(final_idx))==False:
    print (set(list(df_coeffs.T.drop(columns = dummy_col).T.index)).difference(set(list(final_idx))))
add=list(set(list(df_coeffs.T.drop(columns = dummy_col).T.index)).difference(set(list(final_idx))))

Physical_Hor_Picture
Diagonal_dominance_Picture
Warm Hue_Picture
happiness
faces
dummy_motivation_event
ikes-1day


In [74]:
fit_measure = dict()
fit_measure["R-Sq"] = model.rsquared
fit_measure["Adj_R-Sq"] = model.rsquared_adj
fit_measure["AIC"]=model.aic
fit_measure["BIC"]=model.bic
fit_measure["Log-Likehood"] = model.llf
pd.DataFrame(fit_measure, index=["Fit Measure"]).T

,Fit Measure
R-Sq,0.606907
Adj_R-Sq,0.565433
AIC,3905.166860
BIC,4626.146978
Log-Likehood,-1815.583430


In [75]:
np.round_(df_coeffs.T.drop(columns = dummy_col).T.loc[final_idx+add].reset_index(), decimals=4)
#np.round_(df_coeffs.T.drop(columns = dummy_col).T.reset_index(), decimals=3)

,index,Coefficients,Std_error,p,vif
0,product_only,-0.1475,0.1012,0.1449,2.0553
1,model_only,0.6558,0.1158,0.0000,2.6914
2,product_model,0.6719,0.1563,0.0000,1.7078
3,Brightness_Picture,-0.7942,0.1757,0.0000,1.8542
4,Colorfulness_Picture,-0.5801,0.1703,0.0007,1.2772
5,Contrast_Picture_z,0.3877,0.1956,0.0474,1.6980
6,len_text_total,0.0008,0.0003,0.0018,2.0062
7,dummy_motivation_offic,1.0911,0.0959,0.0000,2.1595
8,posts,-0.1701,0.0276,0.0000,1.6834
9,followers,0.0050,0.0005,0.0000,1.1933


In [18]:
regression_vif = df_coeffs.T.drop(columns = dummy_col).T.sort_values(["p"], axis=0, ascending=True).style.bar(subset = ["Coefficients", "p", "vif"],align="left",color=['#d65f5f', '#5fba7d'])
regression_vif

C:\Users\USER\anaconda3\lib\site-packages\pandas\io\formats\style.py:1357: RuntimeWarning: invalid value encountered in true_divide
  normed = width * (s.to_numpy(dtype=float) - smin) / (smax - smin + 1e-12)


,Coefficients,Std_error,p,vif
dummy_motivation_offic,0.962348,0.091523,0.000000,2.017543
followers,0.005138,0.000506,0.000000,1.189622
const,2.915079,0.382900,0.000000,inf
following,0.270421,0.035804,0.000000,1.949465
posts,-0.171744,0.026045,0.000000,1.671654
model_only,0.478683,0.081279,0.000000,2.655674
numBrand,-20.682580,3.618342,0.000000,2.071936
numFocalPd,-12.871668,2.687554,0.000002,1.854356
product_model,0.414951,0.106009,0.000091,1.647553
numModel,7.729081,2.277000,0.000688,2.828361


In [50]:
#df_coeffs.T.drop(columns = dummy_col).T.to_excel("M2_Summary_Statistic.xlsx")

# NBD Regression

In [82]:
x_nbd = x.drop(columns=['const'])
y_nbd = np.exp(y)
model= smf.glm(formula = "y_nbd ~ x_nbd", data=Data6, 
               family=sm.families.NegativeBinomial()).fit(cov_type='HC0')


In [83]:
df_coeffs = pd.DataFrame({"Coefficients": model.params, "p": model.pvalues})
df_coeffs.index=["const"]+[a for a in x_nbd.columns]
df_coeffs["vif"] = vif 
dummy_col = [x for x in df_coeffs.T.columns if re.search("dummy",x) and not(re.search("motivation",x))]
regression_vif = df_coeffs.T.drop(columns = dummy_col).T.sort_values(["vif"], axis=0, ascending=False).style.bar(subset = ["Coefficients", "p", "vif"], align='mid', color=['#d65f5f', '#5fba7d'])

In [84]:
tidy_index = list(pd.read_excel("index_smanalysis.xlsx").dropna()["var"])
final_idx = tidy_index.copy()

In [85]:
counter = 0
for i,idx in enumerate(tidy_index):
    try:
        #print(i)
        df_coeffs.T.drop(columns = dummy_col).T.loc[idx]
    except:
        print(idx)
        final_idx.pop(i-counter)
        counter = counter+1
        
#len(final_idx)==len(tidy_index)
if (len(list(df_coeffs.T.drop(columns = dummy_col).T.index)) == len(final_idx))==False:
    print (set(list(df_coeffs.T.drop(columns = dummy_col).T.index)).difference(set(list(final_idx))))
add=list(set(list(df_coeffs.T.drop(columns = dummy_col).T.index)).difference(set(list(final_idx))))

Physical_Hor_Picture
Diagonal_dominance_Picture
Warm Hue_Picture
happiness
faces
dummy_motivation_event
ikes-1day


In [86]:
fit_measure = dict()
#fit_measure["R-Sq"] = model.rsquared
#fit_measure["Adj_R-Sq"] = model.rsquared_adj
#fit_measure["AIC"]=model.aic
#fit_measure["BIC"]=model.bic
fit_measure["Log-Likehood"] = model.llf
pd.DataFrame(fit_measure, index=["Fit Measure"]).T

,Fit Measure
Log-Likehood,-7568.000446


In [68]:
np.round_(df_coeffs.T.drop(columns = dummy_col).T.loc[final_idx+add], decimals=6).reset_index()

,index,Coefficients,p,vif
0,product_only,-0.058694,0.381830,2.055264
1,model_only,0.426972,0.000000,2.691399
2,product_model,0.265777,0.009180,1.707817
3,Brightness_Picture,-0.726443,0.000023,1.854159
4,Colorfulness_Picture,-0.592389,0.001008,1.277167
5,Contrast_Picture_z,0.299467,0.147598,1.697957
6,len_text_total,0.001517,0.000000,2.006207
7,dummy_motivation_offic,0.778867,0.000000,2.159549
8,posts,-0.159205,0.000000,1.683397
9,followers,0.013847,0.043933,1.193282


In [69]:
regression_vif = df_coeffs.T.drop(columns = dummy_col).T.sort_values(["p"], axis=0, ascending=True).style.bar(subset = ["Coefficients", "p", "vif"],align="left",color=['#d65f5f', '#5fba7d'])
regression_vif

C:\Users\kamilahaniyah\anaconda3\lib\site-packages\pandas\io\formats\style.py:1219: RuntimeWarning: invalid value encountered in true_divide
  normed = width * (s.to_numpy(dtype=float) - smin) / (smax - smin + 1e-12)


,Coefficients,p,vif
const,3.965101,0.000000,inf
dummy_motivation_offic,0.778867,0.000000,2.159549
posts,-0.159205,0.000000,1.683397
following,0.240003,0.000000,1.969731
model_only,0.426972,0.000000,2.691399
len_text_total,0.001517,0.000000,2.006207
numFocalPd,-12.176650,0.000014,1.914382
Brightness_Picture,-0.726443,0.000023,1.854159
Colorfulness_Picture,-0.592389,0.001008,1.277167
numBrand,-10.260306,0.001529,2.116664


# log_comment regression

In [14]:
stop

NameError: name 'stop' is not defined

In [ ]:
UU
Data7 = Data2
y = Data7['log_comment']
x = Data7.drop(["comment","Week","taken_at_timestamp",'log_comment','log_likes','post_age',"smile","happiness"]+log_IV, axis=1).astype(float)
x = sm.add_constant(x)
model= sm.OLS(y, x).fit()

In [ ]:
vif = variance_inflation_factor(x)

df_coeffs = pd.DataFrame({"Coefficients": model.params, "p": model.pvalues, "vif": vif["vif"]})
df_coeffs_sorted = df_coeffs.reindex(df_coeffs["Coefficients"].sort_index(ascending=False).index)
dummy_col = [x for x in df_coeffs.T.columns if re.search("dummy",x) and not(re.search("motivation",x))]
regression_vif = df_coeffs.T.drop(columns = dummy_col).T.sort_values(["vif"], axis=0, ascending=False).style.bar(subset = ["Coefficients", "p", "vif"], align='mid', color=['#d65f5f', '#5fba7d'])

In [ ]:
np.round_(df_coeffs.T.drop(columns = dummy_col).T.loc[tidy_index], decimals=4)

In [ ]:
regression_vif = df_coeffs.T.drop(columns = dummy_col).T.sort_values(["p"], axis=0, ascending=True).style.bar(subset = ["Coefficients", "p", "vif"],align="left",color=['#d65f5f', '#5fba7d'])
regression_vif

# FOR Robust regression in R


In [ ]:
Data_r = Data6.drop(columns=["happiness",])
ori_pdcode_d = [x for x in Data_r.columns if re.search("Hour",x)]
dummy_Week  = [x for x in Data_r.columns if re.search("Week",x)]
dummy_Days  = [x for x in Data_r.columns if re.search("Days",x)]
Data_r["other_Days"] = Data_r.loc[:,dummy_Days ].max(axis=1)
Data_r["other_Days"] = Data_r["other_Days"].apply(lambda x:0 if x == 1 else 1)
Data_r["other_Week"] = Data_r.loc[:,dummy_Week].max(axis=1)
Data_r["other_Week"] = Data_r["other_Week"].apply(lambda x:0 if x == 1 else 1)
Data_r["other_Hour"] = Data_r.loc[:,ori_pdcode_d].max(axis=1)
Data_r["other_Hour"] = Data_r["other_Hour"].apply(lambda x:0 if x == 1 else 1)
Data_r["Week"] = Data_r.loc[:,dummy_Week+["other_Week"]].idxmax(axis=1)
Data_r["Hour"] = Data_r.loc[:,ori_pdcode_d+["other_Hour"]].idxmax(axis=1)
Data_r["Days"] = Data_r.loc[:,dummy_Days+["other_Days"]].idxmax(axis=1)

In [ ]:
#for regression in R
oyof_r = Data_r.drop(columns=ori_pdcode_d+dummy_Week+dummy_Days+["other_Days","other_Week","other_Hour"])
#oyof_r["log_likes"] = Data6['log_likes']
oyof_r.to_excel("SM_R_reg_nohappiness_nonummod_20201104-tv.xlsx", index=False)

In [ ]:
oyof_r.columns